### --- Day 20: Grove Positioning System ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2022/day/20

In [2]:
#!import ../Utils.ipynb

In [3]:
var inputLines = LoadPuzzleInput(2022, 20);
WriteLines(inputLines);

Loading puzzle file: Day20.txt
Total lines: 5000
Max line length: 5

8359
-9175
6992
9836
2244


In [4]:
string[] testInputLines = [
    "1",
    "2",
    "-3",
    "3",
    "-2",
    "0",
    "4",
];

The initial challenge with this puzzle is the need to mix the elements according to their original positions, which means we need to track the elements as they move, so we know which element each mixing instruction is referring to. The second and greater challenge is that we need to move and shuffle elements in the list during mixing. Moving an element between positions $a$ and $b$ potentially means that all elements between these positions need to be shuffled across, depending on our chosen data structure.

If we use a regular list, when we move an element from $a$ to $b$, we need to move every element between $a$ and $b$ also.

If we used a linked list, our movement from $a$ to $b$ is a single update, but the tradeoff is that we lose the ability to index into the list. So finding node $b$ from $a$ requires us to traverse the links in the list.

Just for funsies, I think we'll try the linked list approach, since it involves fewer writes.

#### _Many moments later..._ ####

Welp, what looked like smooth sailing turned into a painful bug hunting exercise! Performance-wise, the linked list traversals were just fine and my code was obtaining the correct answer for the training input. But for some reason it failed to obtain the correct answer with the real input! This is the worst-case scenario because now we don't have anything to compare against :(

In the end, I had to admit defeat and look at [Peter Norvig's solution](https://github.com/norvig/pytudes/blob/main/ipynb/Advent-2022.ipynb) to see what I'd missed. Peter implemented a list-based solution. I reimplemented his solution just so I could compare step-by-step and find what I was missing. Eventually I noticed the subtlety: when we are shuffling element $a$ by a number so large that it wraps around past itself again, we must not count $a$ again as we go past! That is, we must exclude $a$ from the shuffle. Peter achieved this by deleting $a$ before calculating the shuffle. Once I implemented an equivalent, I finally achieved the correct result.

That is, the list `[a, 5, c]`, when moving `5` forward five places, we can imagine our list as an infinite cyclic sequence, but my original interpration of the following was incorrect:

```
[a, 5, c, a, 5, c, a, 5, c...]
    ^--------------^
```

This would suggest we need to move `5` after `a`, giving `[a, 5, c]` again. But this is incorrect!

Instead we should not repeat the `5`, and imagine the sequence like so:

```
[a, 5, c, a, c, a, c, a, c...]
    ^--------------^
```

This means we move the `5` after the `c`, giving us the correct answer of `[a, c, 5]`.

In [5]:
using MovePair = (long, SCG.LinkedListNode<long>);

IEnumerable<IList<long>> DoSomeMixing(string[] inputLines)
{
    long[] initial = inputLines.Select(long.Parse).ToArray();
    LinkedList<long> mixed = new(initial);
    var movePairs = initial.Zip(mixed.WalkNodes()).ToList();

    return MoveOneRound(mixed, movePairs);
}

IEnumerable<IList<long>> MoveOneRound(LinkedList<long> mixed, IList<MovePair> movePairs)
{
    foreach (var (i, node) in movePairs)
    {
        var moves = i % (mixed.Count - 1);
        if (moves is 0) 
        {
            yield return mixed.ToList();
            continue;
        }

        var target = find(moves, node);
        mixed.Remove(node);
        if (moves < 0) { mixed.AddBefore(target, node); }
        else { mixed.AddAfter(target, node); }

        yield return mixed.ToList();
    }

    LinkedListNode<long> find(long dir, LinkedListNode<long> from)
    {
        bool isForwards = dir > 0;
        var steps = Math.Abs(dir);

        var current = from;
        for (var i = 0; i < steps; i++)
        {
            current = isForwards ? current.Next : current.Previous;

            if (current is null)
            {
                current = isForwards ? mixed.First : mixed.Last;
            }

            if (current == from)
            {
                // Super subtle bug fix here! If we wrap around, we should not
                // include the from node again
                i--;
            }
        }

        return current;
    }
}

In [6]:
string render(IEnumerable<long> list) => string.Join(", ", list);

// Initial arrangement:
// 1, 2, -3, 3, -2, 0, 4

// We should expect the following sequence:
// 2, 1, -3, 3, -2, 0, 4
// 1, -3, 2, 3, -2, 0, 4
// 1, 2, 3, -2, -3, 0, 4
// 1, 2, -2, -3, 0, 3, 4
// 1, 2, -3, 0, 3, 4, -2
// 1, 2, -3, 0, 3, 4, -2
// 1, 2, -3, 4, 0, 3, -2

foreach (var mixed in DoSomeMixing(testInputLines))
{
    Console.WriteLine(render(mixed));
}

2, 1, -3, 3, -2, 0, 4
1, -3, 2, 3, -2, 0, 4
1, 2, 3, -2, -3, 0, 4
1, 2, -2, -3, 0, 3, 4
-2, 1, 2, -3, 0, 3, 4
-2, 1, 2, -3, 0, 3, 4
-2, 1, 2, -3, 4, 0, 3


At first glance the comparison looks different, but it's just a difference of opinion on whether we have inserted items at the end or start of the list. When viewed as a sequence of numbers from `0`, the sequence is identical.

In [7]:
// Here is the Peter Norvig solution I used for comparison:
// Source: https://github.com/norvig/pytudes/blob/main/ipynb/Advent-2022.ipynb
IEnumerable<IList<long>> Mix2(string[] inputLines)
{
    var numbers = inputLines.Select(long.Parse).ToArray();
    var N = numbers.Length;
    var mixedNums = Range(0, N).Select(i => (long)i).ToList();
    foreach (long n in Range(0, N))
    {
        var i = mixedNums.IndexOf(n);
        mixedNums.RemoveAt(i);
        var j = (i + numbers[n]) % (N - 1);
        j += j > 0 ? 0 : (N - 1);
        var jInt = (int)j;
        mixedNums.Insert(jInt, n);

        // Return each list so we can compare!
        var mixed = mixedNums.Select(i => numbers[i]).ToList();
        yield return mixed;
    }
}

foreach (var mixed in Mix2(testInputLines))
{
    Console.WriteLine(render(mixed));
}

2, 1, -3, 3, -2, 0, 4
1, -3, 2, 3, -2, 0, 4
1, 2, 3, -2, -3, 0, 4
1, 2, -2, -3, 0, 3, 4
1, 2, -3, 0, 3, 4, -2
1, 2, -3, 0, 3, 4, -2
1, 2, -3, 4, 0, 3, -2


In [8]:
// Reset the sequence to start from 0, so we can compare
IList<long> Zeroify(IList<long> source)
{
    var N = source.Count;
    var zeroStart = source.IndexOf(0);
    var zeroMixed = Range(0, N).Select(i => source[(i + zeroStart) % N]).ToList();
    return zeroMixed;
}

// Then, the grove coordinates can be found by looking at the 1000th, 2000th,
// and 3000th numbers after the value 0, wrapping around the list as necessary.
long MixAnswer(IEnumerable<IList<long>> mixedStages)
{
    var mixed = mixedStages.Last();
    mixed = Zeroify(mixed);

    int[] inspections = [1000, 2000, 3000];
    var N = mixed.Count;

    return inspections.Select(i => mixed[i % N]).Sum();
}

In [9]:
// In the above example, the 1000th number after 0 is 4, the 2000th is -3, and
// the 3000th is 2; adding these together produces 3.

var demoAnswer = MixAnswer(Mix2(testInputLines));
Console.WriteLine($"Demo answer: {demoAnswer}");

var testAnswer = MixAnswer(DoSomeMixing(testInputLines));
Console.WriteLine($"Test answer: {testAnswer}");

Demo answer: 3
Test answer: 3


In [10]:
// The item-by-item comparison that helped us find our bug
void Compare(string[] inputLines)
{
    var mine = DoSomeMixing(inputLines).Select(Zeroify);
    var demo = Mix2(inputLines).Select(Zeroify);

    foreach (var (m, d) in mine.Zip(demo))
    {
        // Console.WriteLine($"Mine: {render(m)}");
        // Console.WriteLine($"Demo: {render(d)}");

        foreach (var (i, (mm, dd)) in m.Zip(d).Index())
        {
            if (mm != dd)
            {
                Console.WriteLine($"Values differ at index {i}!");
                Console.WriteLine($"Mine: {mm}. Theirs: {dd}");
                throw new Exception("Stopping here");
            }
        }
    }
}

Compare(testInputLines);
Compare(inputLines);

In [11]:
// Mix your encrypted file exactly once. What is the sum of the three numbers
// that form the grove coordinates?

var part1Answer = MixAnswer(DoSomeMixing(inputLines));
Console.WriteLine(part1Answer);

23321


In [12]:
// 23321 is correct!
Ensure(23321, part1Answer);

### --- Part Two ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2022/day/20

Ok, for part 2 it looks like we should be able to achieve this with some simple modifications to part 1. Multiplying each item by the decryption key would result in huge traversals, especially for our linked list, but this is mitigated since we calculate our movements modulo list size.

In [14]:
const long DecryptionKey = 811589153;

IEnumerable<IList<long>> DoSomeMixing2(string[] inputLines, long multiplier = DecryptionKey, int rounds = 10)
{
    var initial = inputLines.Select(long.Parse).ToArray();
    initial = initial.Select(i => i * multiplier).ToArray();
    LinkedList<long> mixed = new(initial);
    var movePairs = initial.Zip(mixed.WalkNodes()).ToList();

    foreach (var _ in Range(0, rounds))
    {
        foreach (var r in MoveOneRound(mixed, movePairs))
        {
            yield return r;
        }
    }
}

In [15]:
// The grove coordinates can still be found in the same way... adding these
// together produces 1623178306.

var part2TestAnswer = MixAnswer(DoSomeMixing2(testInputLines, multiplier: DecryptionKey, rounds: 10));
Console.WriteLine(part2TestAnswer);

1623178306


In [16]:
// Apply the decryption key and mix your encrypted file ten times. What is the
// sum of the three numbers that form the grove coordinates?

var part2Answer = MixAnswer(DoSomeMixing2(inputLines, multiplier: DecryptionKey, rounds: 10));
Console.WriteLine(part2Answer);

1428396909280


In [17]:
// 1428396909280 is correct!
Ensure(1428396909280, part2Answer);